# Step 0 - Destiny 2 Stats - Setup

En este cuaderno están las funcionalidades básicas para configurar el acceso al API.
Este cuaderno genera una serie de ficheros en la carpeta de configuración con la información del usuario y el personaje que serán leidos desde otros cuadernos para obtener la información.

In [1]:
import requests
import json
import os
import pandas as pd
import base64
import urllib
from calendar import monthrange
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def readAPIKeyFromFile():
    apikeyfile = open("apikey.txt","r+")
    apikey = apikeyfile.readline()
    apikeyfile.close
    return apikey

## Especificar el APIKey y el nombre del jugador

Indicar a continuación el __nombre del jugador__ que se va a analizar (al cual pertenecen los personajes) y el API Key que hemos obtenido desde la web de Bungie (Hay más instrucciones en el [fichero README de este repositorio](https://github.com/findemor/destiny-2-data-science/blob/main/README.md))



In [3]:
# Parametros
ApiKey = readAPIKeyFromFile() #o mete aqui tu API Key directamente asi ApiKey = "TU_API_KEY"
username = "findemor"

### A continuación se definen algunas funciones necesarias más adelante

In [4]:
# Carpeta auxiliar para almacenar los ficheros temporales
def requestWithCache(method, url, headers, data, verbose = False):
    cacheFolder = 'DestinyTemp'
    cacheFile = 'cache-0.json'

    if not os.path.exists(cacheFolder):
        os.makedirs(cacheFolder)
    
    cachePath = cacheFolder + "/" + cacheFile
    
    #recuperamos la cache
    cache = {}
    if os.path.isfile(cachePath):
        cachef = open(cachePath,"r+")
        cached = cachef.readlines()
        if len(cached) > 0:
            cache = json.loads(cached[0])
        cachef.close()    
    
    r = ""
    
    if method == "GET":
        cachekey = url #base64.urlsafe_b64encode(bytes(url, 'utf-8')).decode('ascii')
        if cachekey in cache:
            if verbose:
                print("cache found " + cachekey)
            r = cache[cachekey]            
        else:
            if verbose:
                print("cache not found " + cachekey)
            response = requests.request(method, url = url, headers=headers, data=data)
            r = response.text
            #grabamos la cache
            cache[cachekey] = r
            cachef = open(cachePath,"w+")
            cachef.write(json.dumps(cache))
            cachef.close()
    else:
        if verbose:
            print("cache not apply")
        r = requests.request(method, url = url, headers=headers, data=data)
    
        
    cachef.close()
    
    return r

In [5]:
def downloadClassDefinition():
    base = "https://www.bungie.net"
    url = base + "/Platform/Destiny2/Manifest/"
    headers = { 'X-API-Key': ApiKey }
    #response = requests.request("GET", url, headers=headers, data={})
    response = requestWithCache(method = "GET", url = url, headers = headers, data = {})
    data = json.loads(response)
    def_url = base + data["Response"]["jsonWorldComponentContentPaths"]["en"]["DestinyClassDefinition"]
    #response = requests.request("GET", url = def_url, headers=headers, data={})
    response = requestWithCache(method = "GET", url = def_url, headers = headers, data = {})
    data = json.loads(response)
    
    # convierto el json en un dataframe
    dfdata = pd.DataFrame(columns=['classHash', 'name'])
    i = 0
    for d in data:
        #for i,c in enumerate(data["Response"]["characters"]["data"][d]):
        c = data[d]
        dfdata.loc[i] = [d, data[d]["displayProperties"]["name"]]
        i += 1
    return dfdata

classesManifest = downloadClassDefinition()
classesManifest.classHash = classesManifest.classHash.astype(str)

In [6]:
# Buscar la cuenta de usuario que queremos analizar
def searchMembershipId(username):
    url = "https://www.bungie.net/Platform/Destiny2/SearchDestinyPlayer/All/"+ username
    headers = { 'X-API-Key': ApiKey }
    #response = requests.request("GET", url, headers=headers, data={})
    response = requestWithCache(method = "GET", url = url, headers = headers, data = {})
    data = json.loads(response)
    
    # convierto el json en un dataframe
    dfdata = pd.DataFrame(columns=['displayName', 'membershipId', 'membershipType'])
    for i,c in enumerate(data["Response"]):
        dfdata.loc[i] = [c["displayName"], c["membershipId"], c['membershipType']]
    return dfdata

In [7]:
# Buscar el personaje que queremos analizar
def searchCharacterId(membershipType, membershipId):
    url = "https://www.bungie.net/Platform/Destiny2/" + membershipType +"/Profile/" + membershipId + "/?components=200"
    headers = { 'X-API-Key': ApiKey }
    #response = requests.request("GET", url, headers=headers, data={})
    response = requestWithCache(method = "GET", url = url, headers = headers, data = {})
    data = json.loads(response)
    
    # convierto el json en un dataframe
    dfdata = pd.DataFrame(columns=['characterId', 'light', 'classHash'])
    i = 0
    for d in data["Response"]["characters"]["data"]:
        #for i,c in enumerate(data["Response"]["characters"]["data"][d]):
        c = data["Response"]["characters"]["data"][d]
        dfdata.loc[i] = [c["characterId"], c["light"], c['classHash']]
        i += 1
    return dfdata

## Comprobamos los personajes del jugador que hemos indicado más arriba

In [8]:
members = searchMembershipId(username)
members

,displayName,membershipId,membershipType
0,findemor,4611686018445251617,2
1,findemor,4611686018445251617,2


## Del cuadro anterior, identificar y establecer los valores de membresia
Observando los dos cuadros anteriores, identificar el membershipId, el membershipType que corresponden al usuario que estamos buscando.

In [9]:
membership = {
    'membershipId': "4611686018445251617",
    'membershipType': "2"
}

In [10]:
characters = searchCharacterId(membershipType = membership["membershipType"], membershipId = membership["membershipId"])
characters.classHash = characters.classHash.astype(str)

decoded_characters = pd.merge(left = characters, right = classesManifest, left_on="classHash", right_on="classHash")
decoded_characters

,characterId,light,classHash,name
0,2305843009279555211,1292,3655393761,Titan
1,2305843009294017733,1320,671679327,Hunter
2,2305843009700575195,1274,2271682572,Warlock


## Del cuadro anterior, identificar y establecer los valores del personaje a analizar

Elegir el que characterId que corresponda y establecer los valores a continuación

In [11]:
character = {
    "characterId": "2305843009294017733"
}

# Se guardan todos estos valores para su uso posterior
Ahora se van a generar los ficheros de la carpeta config con la informacion que hemos introducido

In [12]:
data = {
    "ApiKey": ApiKey,
    "username": username,
    "character": character,
    "membership": membership
}

folder = "config"

if not os.path.exists(folder):
    os.makedirs(folder)

cachef = open(folder + "/setup.json","w+")
cachef.write(json.dumps(data))
cachef.close()

## Abrir el siguiente Notebook